In [1]:
eventDF = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferSchema='true').load('/mnt/S3/data/dataSetEvents.csv')

In [2]:
eventDF.show(3)

In [3]:
import copy

def event_split(x):
  y=copy.deepcopy(x).asDict()
  event=y['event'].replace("_"," ").split()
  y['event_action'] = event[0]
  target_string = event[1:len(event)]
  y['event_target'] = (" ".join(str(x) for x in target_string))
  return y

In [4]:
event_details = eventDF.select("event").rdd.map(event_split).toDF().distinct()
event_details.show(3)

In [5]:
newDF = eventDF.join(event_details, on = 'event', how = 'inner').distinct()
#newDF.select("event").distinct().show()
#check it's right

In [6]:
#event split into event_action and event_target
newDF.show(3)

In [7]:
import pyspark.sql.functions

split_col = pyspark.sql.functions.split(newDF['timestamp'], ' ')
newDF = newDF.withColumn('date', split_col.getItem(0))
newDF = newDF.withColumn('time', split_col.getItem(1))
newDF.show(3)

#newDF now has columns for date and time

In [8]:
#A VIN prefix is taken from a VIN, specifically characters at positions 1 thru 8 and 10

from pyspark.sql.functions import concat, lit

newerDF=newDF.withColumn('vin_prefix', concat(newDF.vin.substr(1, 8), lit('_'), newDF.vin.substr(10, 1)))
newerDF.show(3)

In [9]:
#display(newerDF)
newerDF.count()

In [10]:
ymmDF = newerDF.select('vin_prefix', 'year', 'make', 'model').distinct().orderBy('make', 'model', 'year', 'vin_prefix')
ymmDF.show(3)


In [11]:
vinDF = newerDF.select('vin', 'vin_prefix', 'trim', 'body_style', 'cab_style').distinct().orderBy('vin')
vinDF.show(3)
vinDF.count()

In [12]:
locDF = newerDF.select('zip', 'city', 'state').distinct().orderBy('zip', 'city', 'state')
locDF.show(3)

In [13]:
listingDF = newerDF.select('vin', 'date', 'zip', 'city', 'state', 'condition', 'price', 'mileage', 'image_count').distinct().orderBy('vin', 'date')
listingDF.show(3)

In [14]:
usereventsDF = newerDF.select('user_id', 'vin', 'date', 'referring_domain', 'event_action', 'event_target', 'time').groupBy('user_id', 'vin', 'date', 'referring_domain', 'event_action', 'event_target', 'time').count().orderBy('user_id', 'vin', 'date', 'time', 'count')
display(usereventsDF) 

In [15]:
adDF = newerDF.select('user_id', 'vin', 'condition', 'price', 'mileage', 'year').groupBy('user_id', 'vin', 'condition', 'price', 'mileage', 'year').count().orderBy('user_id', 'vin', 'condition', 'price', 'count')
adDF.show(3)


In [16]:
# ymmDF.repartition(1).write.format('com.databricks.spark.csv').options(header='true').save('/mnt/S3/output/Finalymm')
# vinDF.repartition(1).write.format('com.databricks.spark.csv').options(header='true').save('/mnt/S3/output/Finalvin')
# locDF.repartition(1).write.format('com.databricks.spark.csv').options(header='true').save('/mnt/S3/output/Finalloc')
# listingDF.repartition(1).write.format('com.databricks.spark.csv').options(header='true').save('/mnt/S3/output/Finallisting')
# usereventsDF.repartition(1).write.format('com.databricks.spark.csv').options(header='true').save('/mnt/S3/output/Finaluserevents')
# adDF.repartition(1).write.format('com.databricks.spark.csv').options(header='true').save('/mnt/S3/output/Finalad')
